# Notebook 1.2: LMPC Objective Function and QP Formulation

In Notebook 1.1, we learned how to predict the future behavior of a linear system. Now, we'll define what constitutes "good" future behavior by formulating an **objective function** (also called a cost function). For Linear MPC (LMPC), this objective function is typically quadratic.

The MPC controller's goal is to find a sequence of future control inputs $\mathbf{U}_k$ that minimizes this objective function, subject to system dynamics (handled by the prediction model) and constraints (which we'll cover in Notebook 1.4).

This minimization problem, for LMPC, can be cast as a **Quadratic Program (QP)**, a type of optimization problem that can be solved efficiently.

**Goals of this Notebook:**
1. Understand the components of a typical quadratic LMPC objective function: tracking error, control effort, and control move suppression.
2. Learn about the role of weighting matrices $\mathbf{Q}$, $\mathbf{R}$, and $\mathbf{S}$.
3. Formulate the LMPC problem into the standard QP form: $\min_{\mathbf{U}_k} \frac{1}{2} \mathbf{U}_k^T \mathbf{H}_{QP} \mathbf{U}_k + \mathbf{f}_{QP}^T \mathbf{U}_k$.
4. Derive and implement the construction of the QP matrices $\mathbf{H}_{QP}$ and $\mathbf{f}_{QP}$ using the prediction matrices $\mathbf{F}$ and $\mathbf{\Phi}$ from Notebook 1.1.
5. Apply this to our double integrator example.

## 1. Importing Libraries and Re-using Code from Notebook 1.1

We'll need NumPy and Matplotlib. We will also reuse the `get_prediction_matrices` function and the double integrator system defined in Notebook 1.1.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import cont2discrete

# Optional: for nicer plots
plt.rcParams.update({'font.size': 12, 'figure.figsize': (8, 5)})

# --- Code from Notebook 1.1 (Double Integrator & Prediction Matrices) ---
Ac = np.array([[0, 1], [0, 0]])
Bc = np.array([[0], [1]])
Cc = np.array([[1, 0]])
Dc = np.array([[0]])
Ts = 0.1
system_continuous = (Ac, Bc, Cc, Dc)
Ad, Bd, Cd, Dd, _ = cont2discrete(system_continuous, Ts, method='zoh')

def get_prediction_matrices(Ad_sys, Bd_sys, Cd_sys, Np_horizon):
    n_states = Ad_sys.shape[0]
    n_inputs = Bd_sys.shape[1]
    n_outputs = Cd_sys.shape[0]

    F = np.zeros((n_outputs * Np_horizon, n_states))
    Phi = np.zeros((n_outputs * Np_horizon, n_inputs * Np_horizon))

    # Populate F matrix: Y_pred[i] = y_k+i+1
    # F = [CA; CA^2; ...; CA^Np]
    for i in range(Np_horizon):
        F[i * n_outputs:(i + 1) * n_outputs, :] = Cd_sys @ np.linalg.matrix_power(Ad_sys, i + 1)

    # Populate Phi matrix (Lower block triangular Toeplitz)
    # Phi_ij (block) = C * A^(i-j) * B for i >= j, 0 otherwise (i,j are 0-indexed block indices)
    # This matches Y_pred[i] (y_k+i+1) containing effect of u_k+j
    # y_k+i+1 = ... + C A^(i-j) B u_k+j + ...
    for i in range(Np_horizon):  # block row for y_k+i+1 (0 to Np-1)
        for j in range(i + 1):  # block col for u_k+j (0 to i)
            if i - j == 0:
                Phi[i*n_outputs:(i+1)*n_outputs, j*n_inputs:(j+1)*n_inputs] = Cd_sys @ Bd_sys
            else:
                Phi[i*n_outputs:(i+1)*n_outputs, j*n_inputs:(j+1)*n_inputs] = Cd_sys @ np.linalg.matrix_power(Ad_sys, i - j) @ Bd_sys
    return F, Phi

# Example system parameters from Notebook 1.1
Np = 10 # Prediction Horizon for MPC
Nc = 10 # Control Horizon (assume Nc=Np for now for simpler Phi in QP)
        # If Nc < Np, Phi needs to be constructed carefully based on how U is defined.
        # For this notebook, we'll assume U contains Np (or Nc) control moves.
        # Let's assume we optimize over Nc control moves, and U in QP is [u_k, ..., u_k+Nc-1]
        # Then Phi needs to be (Np*p x Nc*m).

# For constructing the QP, it's often simpler if Phi corresponds to the length of U decision vector.
# Let's define Phi based on Nc for the control inputs that are decision variables.
# The predictions Y still go up to Np.
# Y_k = Fx_k + Phi_truncated * U_truncated_k + Phi_fixed * U_fixed_k
# For now, let's stick to the case where decision variables U cover all inputs influencing Y up to Np.
# So U will have Np*m elements if Dd=0, or slightly more if Dd is not zero and inputs affect outputs instantaneously.
# We will use Phi for U_k = [u_k|k, ..., u_k+Np-1|k]
F_matrix, Phi_matrix = get_prediction_matrices(Ad, Bd, Cd, Np)

print(f"Ad shape: {Ad.shape}, Bd shape: {Bd.shape}, Cd shape: {Cd.shape}")
print(f"Np = {Np}")
print(f"F_matrix shape: {F_matrix.shape}")
print(f"Phi_matrix shape: {Phi_matrix.shape}")

Ad shape: (2, 2), Bd shape: (2, 1), Cd shape: (1, 2)
Np = 10
F_matrix shape: (10, 2)
Phi_matrix shape: (10, 10)


## 2. The LMPC Objective Function

A common quadratic objective function for LMPC is (from Chapter 2, Eq. 2.3):

$$ J(\mathbf{U}_k, x_k) = \sum_{j=1}^{N_p} || \hat{y}_{k+j|k} - r_{k+j} ||^2_{\mathbf{Q}_j} + \sum_{j=0}^{N_c-1} || u_{k+j|k} ||^2_{\mathbf{R}_j} + \sum_{j=0}^{N_c-1} || \Delta u_{k+j|k} ||^2_{\mathbf{S}_j} $$

Where:
- $\hat{y}_{k+j|k}$: Predicted output at future time $k+j$, based on information at time $k$.
- $r_{k+j}$: Reference or setpoint for the output at $k+j$.
- $u_{k+j|k}$: Control input at $k+j$, decided at time $k$.
- $\Delta u_{k+j|k} = u_{k+j|k} - u_{k+j-1|k}$ (with $u_{k-1|k}$ being $u_{k-1}$, the previously applied input).
- $N_p$: Prediction horizon.
- $N_c$: Control horizon ($N_c \le N_p$). The number of future control moves that are distinct optimization variables. Beyond $N_c$, inputs might be held constant: $u_{k+j|k} = u_{k+N_c-1|k}$ for $j \ge N_c$.
- $\mathbf{Q}_j \ge 0$: Weighting matrix for output tracking error (p x p).
- $\mathbf{R}_j > 0$: Weighting matrix for control input magnitude (m x m).
- $\mathbf{S}_j \ge 0$: Weighting matrix for control input rate of change (m x m).

For simplicity in this notebook, we will initially assume:
1. $N_c = N_p$. This means our optimization vector $\mathbf{U}_k$ will contain $N_p$ control moves: $\mathbf{U}_k = [u_{k|k}^T, u_{k+1|k}^T, \dots, u_{k+N_p-1|k}^T]^T$.
2. $\mathbf{Q}_j = \mathbf{Q}$, $\mathbf{R}_j = \mathbf{R}$, $\mathbf{S}_j = \mathbf{S}$ (constant weights over the horizon).
3. The output $y_k$ has $p$ elements, and input $u_k$ has $m$ elements.

### Vectorized Form of the Objective Function

Let:
- $\mathbf{Y}_k = \mathbf{F} x_k + \mathbf{\Phi} \mathbf{U}_k$ be the vector of $N_p$ predicted outputs.
- $\mathbf{R}_{traj} = [r_{k+1}^T, r_{k+2}^T, \dots, r_{k+N_p}^T]^T$ be the vector of future reference setpoints.
- $\bar{\mathbf{Q}} = \text{diag}(\mathbf{Q}, \mathbf{Q}, \dots, \mathbf{Q})$ ($N_p$ times, total size $N_p p \times N_p p$).
- $\bar{\mathbf{R}} = \text{diag}(\mathbf{R}, \mathbf{R}, \dots, \mathbf{R})$ ($N_p$ times, total size $N_p m \times N_p m$).
- $\bar{\mathbf{S}} = \text{diag}(\mathbf{S}, \mathbf{S}, \dots, \mathbf{S})$ ($N_p$ times, total size $N_p m \times N_p m$).

The objective function can be written as:
$$ J = (\mathbf{Y}_k - \mathbf{R}_{traj})^T \bar{\mathbf{Q}} (\mathbf{Y}_k - \mathbf{R}_{traj}) + \mathbf{U}_k^T \bar{\mathbf{R}} \mathbf{U}_k + \Delta \mathbf{U}_k^T \bar{\mathbf{S}} \Delta \mathbf{U}_k $$

To handle $\Delta \mathbf{U}_k$, we define $\Delta u_{k|k} = u_{k|k} - u_{k-1}$ and $\Delta u_{k+j|k} = u_{k+j|k} - u_{k+j-1|k}$ for $j > 0$.
We can write $\Delta \mathbf{U}_k = \mathbf{T}_{diff} \mathbf{U}_k - \mathbf{U}_{prev}$, where $\mathbf{T}_{diff}$ is a differencing matrix and $\mathbf{U}_{prev}$ incorporates $u_{k-1}$.
$\mathbf{T}_{diff} = \begin{bmatrix} I & 0 & \dots & 0 \\ -I & I & \dots & 0 \\ \vdots & \ddots & \ddots & \vdots \\ 0 & \dots & -I & I \end{bmatrix}$ (size $N_p m \times N_p m$)
$\mathbf{U}_{prev} = \begin{bmatrix} u_{k-1} \\ 0 \\ \vdots \\ 0 \end{bmatrix}$ (size $N_p m \times 1$)

So the term $\Delta \mathbf{U}_k^T \bar{\mathbf{S}} \Delta \mathbf{U}_k = (\mathbf{T}_{diff} \mathbf{U}_k - \mathbf{U}_{prev})^T \bar{\mathbf{S}} (\mathbf{T}_{diff} \mathbf{U}_k - \mathbf{U}_{prev})$.

## 3. Deriving the QP Matrices $\mathbf{H}_{QP}$ and $\mathbf{f}_{QP}$

We want to express $J$ in the standard QP form: $\frac{1}{2} \mathbf{U}_k^T \mathbf{H}_{QP} \mathbf{U}_k + \mathbf{f}_{QP}^T \mathbf{U}_k + \text{constant terms}$.
(The constant terms do not affect the optimal $\mathbf{U}_k$ and can be ignored for finding the minimizer).

Substitute $\mathbf{Y}_k = \mathbf{F} x_k + \mathbf{\Phi} \mathbf{U}_k$ into the tracking term:
$J_{track} = (\mathbf{\Phi} \mathbf{U}_k + \mathbf{F} x_k - \mathbf{R}_{traj})^T \bar{\mathbf{Q}} (\mathbf{\Phi} \mathbf{U}_k + \mathbf{F} x_k - \mathbf{R}_{traj})$
$J_{track} = \mathbf{U}_k^T \mathbf{\Phi}^T \bar{\mathbf{Q}} \mathbf{\Phi} \mathbf{U}_k + 2 (\mathbf{F} x_k - \mathbf{R}_{traj})^T \bar{\mathbf{Q}} \mathbf{\Phi} \mathbf{U}_k + \text{const.}$ 

The control effort term is already in quadratic form: $J_{effort} = \mathbf{U}_k^T \bar{\mathbf{R}} \mathbf{U}_k$.

The control move suppression term: 
$J_{move} = (\mathbf{T}_{diff} \mathbf{U}_k - \mathbf{U}_{prev})^T \bar{\mathbf{S}} (\mathbf{T}_{diff} \mathbf{U}_k - \mathbf{U}_{prev})$
$J_{move} = \mathbf{U}_k^T \mathbf{T}_{diff}^T \bar{\mathbf{S}} \mathbf{T}_{diff} \mathbf{U}_k - 2 \mathbf{U}_{prev}^T \bar{\mathbf{S}} \mathbf{T}_{diff} \mathbf{U}_k + \text{const.}$

Combining all terms quadratic in $\mathbf{U}_k$ and linear in $\mathbf{U}_k$:

$$ \mathbf{H}_{QP} = 2 \left( \mathbf{\Phi}^T \bar{\mathbf{Q}} \mathbf{\Phi} + \bar{\mathbf{R}} + \mathbf{T}_{diff}^T \bar{\mathbf{S}} \mathbf{T}_{diff} \right) $$

$$ \mathbf{f}_{QP}^T = 2 \left( (\mathbf{F} x_k - \mathbf{R}_{traj})^T \bar{\mathbf{Q}} \mathbf{\Phi} - \mathbf{U}_{prev}^T \bar{\mathbf{S}} \mathbf{T}_{diff} \right) $$
or
$$ \mathbf{f}_{QP} = 2 \left( \mathbf{\Phi}^T \bar{\mathbf{Q}} (\mathbf{F} x_k - \mathbf{R}_{traj}) - \mathbf{T}_{diff}^T \bar{\mathbf{S}} \mathbf{U}_{prev} \right) $$

Note: The factor of 2 comes from matching $J = \mathbf{U}^T M \mathbf{U} + 2 L^T \mathbf{U}$ with $\frac{1}{2} \mathbf{U}^T \mathbf{H} \mathbf{U} + \mathbf{f}^T \mathbf{U}$, so $\mathbf{H} = 2M$ and $\mathbf{f} = 2L$ if we expand $(X+Y)^2 = X^2 + 2XY + Y^2$. Or simply $\mathbf{H} = M$ and $\mathbf{f} = L$ if we directly take the Hessian and gradient of J and the QP is defined as $\min \mathbf{x}^T H \mathbf{x} + \mathbf{f}^T \mathbf{x}$. Many QP solvers expect the $rac{1}{2}$ in the objective, so $H_{QP}$ is the Hessian itself.

Let's redefine slightly for the standard QP form $\min_{\mathbf{z}} \frac{1}{2} \mathbf{z}^T P \mathbf{z} + \mathbf{q}^T \mathbf{z}$ (using $P, q$ to avoid confusion with our state $x$).
Here $\mathbf{z} = \mathbf{U}_k$.

From $J_{track} = (\mathbf{\Phi U} + \mathbf{E})^T \bar{\mathbf{Q}} (\mathbf{\Phi U} + \mathbf{E})$ where $\mathbf{E} = \mathbf{F} x_k - \mathbf{R}_{traj}$.
$J_{track} = \mathbf{U}^T \mathbf{\Phi}^T \bar{\mathbf{Q}} \mathbf{\Phi U} + 2 \mathbf{E}^T \bar{\mathbf{Q}} \mathbf{\Phi U} + \mathbf{E}^T \bar{\mathbf{Q}} \mathbf{E}$.

From $J_{move} = (\mathbf{T}_{diff} \mathbf{U} - \mathbf{U}_{prev})^T \bar{\mathbf{S}} (\mathbf{T}_{diff} \mathbf{U} - \mathbf{U}_{prev})$.
$J_{move} = \mathbf{U}^T \mathbf{T}_{diff}^T \bar{\mathbf{S}} \mathbf{T}_{diff} \mathbf{U} - 2 \mathbf{U}_{prev}^T \bar{\mathbf{S}} \mathbf{T}_{diff} \mathbf{U} + \mathbf{U}_{prev}^T \bar{\mathbf{S}} \mathbf{U}_{prev}$.

So, the quadratic term for $\mathbf{U}$ is $\mathbf{U}^T (\mathbf{\Phi}^T \bar{\mathbf{Q}} \mathbf{\Phi} + \bar{\mathbf{R}} + \mathbf{T}_{diff}^T \bar{\mathbf{S}} \mathbf{T}_{diff}) \mathbf{U}$.
The linear term for $\mathbf{U}$ is $2 ( (\mathbf{F} x_k - \mathbf{R}_{traj})^T \bar{\mathbf{Q}} \mathbf{\Phi} - \mathbf{U}_{prev}^T \bar{\mathbf{S}} \mathbf{T}_{diff} ) \mathbf{U}$.

Therefore, for the QP form $\frac{1}{2} \mathbf{U}^T \mathbf{H}_{QP} \mathbf{U} + \mathbf{f}_{QP}^T \mathbf{U}$:
$$ \mathbf{H}_{QP} = 2 \left( \mathbf{\Phi}^T \bar{\mathbf{Q}} \mathbf{\Phi} + \bar{\mathbf{R}} + \mathbf{T}_{diff}^T \bar{\mathbf{S}} \mathbf{T}_{diff} \right) $$
$$ \mathbf{f}_{QP} = 2 \left( \mathbf{\Phi}^T \bar{\mathbf{Q}} (\mathbf{F} x_k - \mathbf{R}_{traj}) - \mathbf{T}_{diff}^T \bar{\mathbf{S}} \mathbf{U}_{prev} \right) $$
This seems consistent. The factor of 2 in $\mathbf{f}_{QP}$ arises from the $2 \mathbf{E}^T ar{\mathbf{Q}} \mathbf{\Phi U}$ term.

In [2]:
def build_qp_matrices(F, Phi, Q_w, R_w, S_w, Np, m_inputs, p_outputs, u_prev):
    """
    Builds the H_qp and f_qp matrices for the QP problem.
    Assumes Nc = Np for simplicity here (U is Np*m long).
    Q_w, R_w, S_w are the scalar weights if Q,R,S are diagonal (Q_val*I, R_val*I, S_val*I)
    or they are the actual (p x p) or (m x m) matrices.
    u_prev is the (m x 1) last applied input.
    """
    # Construct block diagonal Q_bar, R_bar, S_bar
    # If Q_w, R_w, S_w are scalars, create identity matrices scaled by them.
    # Otherwise, assume they are the matrices Q, R, S.
    
    if np.isscalar(Q_w):
        Q_mat = Q_w * np.eye(p_outputs)
    else:
        Q_mat = Q_w
        
    if np.isscalar(R_w):
        R_mat = R_w * np.eye(m_inputs)
    else:
        R_mat = R_w
        
    if np.isscalar(S_w):
        S_mat = S_w * np.eye(m_inputs)
    else:
        S_mat = S_w

    Q_bar = np.kron(np.eye(Np), Q_mat) # Np*p x Np*p
    R_bar = np.kron(np.eye(Np), R_mat) # Np*m x Np*m
    S_bar = np.kron(np.eye(Np), S_mat) # Np*m x Np*m
    
    # Construct T_diff matrix for Delta_U
    # Delta_U_k = T_diff * U_k - U_prev_stacked
    # T_diff is (Np*m x Np*m)
    T_diff = np.zeros((Np * m_inputs, Np * m_inputs))
    for i in range(Np):
        T_diff[i*m_inputs:(i+1)*m_inputs, i*m_inputs:(i+1)*m_inputs] = np.eye(m_inputs)
        if i > 0:
            T_diff[i*m_inputs:(i+1)*m_inputs, (i-1)*m_inputs:i*m_inputs] = -np.eye(m_inputs)
            
    # Construct U_prev_stacked for Delta_U term
    # U_prev_stacked = [u_prev; 0; ...; 0]
    U_prev_stacked = np.zeros((Np * m_inputs, 1))
    U_prev_stacked[0:m_inputs, 0] = u_prev.flatten()
    
    # Calculate H_qp
    # H_qp = 2 * (Phi^T Q_bar Phi + R_bar + T_diff^T S_bar T_diff)
    H_qp = 2 * (Phi.T @ Q_bar @ Phi + R_bar + T_diff.T @ S_bar @ T_diff)
    
    # f_qp depends on current state x_k and reference R_traj
    # f_qp = 2 * (Phi^T Q_bar (F x_k - R_traj) - T_diff^T S_bar U_prev_stacked)
    # The function will return a lambda to compute f_qp given x_k and R_traj
    
    def compute_f_qp(xk, R_traj_vector):
        # R_traj_vector should be (Np*p x 1)
        # xk should be (n_states x 1)
        error_term = F @ xk - R_traj_vector
        f_qp = 2 * (Phi.T @ Q_bar @ error_term - T_diff.T @ S_bar @ U_prev_stacked)
        return f_qp
        
    return H_qp, compute_f_qp

# --- Example for Double Integrator ---
num_states_di = Ad.shape[0]
num_inputs_di = Bd.shape[1]
num_outputs_di = Cd.shape[0]

# Define weights (scalars for simplicity, meaning Q = Qw*I, R = Rw*I, S = Sw*I)
Qw = 10.0  # Weight on output tracking error (position)
Rw = 0.1   # Weight on control input magnitude (force)
Sw = 0.5   # Weight on control input rate of change

u_prev_example = np.array([[0.0]]) # Initially, previous input was 0

H_qp_di, compute_f_qp_di = build_qp_matrices(
    F_matrix, Phi_matrix, 
    Qw, Rw, Sw, 
    Np, num_inputs_di, num_outputs_di, 
    u_prev_example
)

print(f"Shape of H_qp_di: {H_qp_di.shape}") # Should be (Np*m x Np*m)

# Define a current state and a reference trajectory for testing f_qp
x_current_di = np.array([[0.5], [-0.1]]) # Example current state (pos=0.5, vel=-0.1)

# Reference trajectory: e.g., reach position 1.0 and stay there
R_traj_di_flat = np.ones((Np * num_outputs_di, 1)) * 1.0 

f_qp_di = compute_f_qp_di(x_current_di, R_traj_di_flat)
print(f"Shape of f_qp_di: {f_qp_di.shape}") # Should be (Np*m x 1)

# Check if H_qp is positive definite (important for unique solution)
try:
    np.linalg.cholesky(H_qp_di) # Cholesky decomposition only works for positive definite matrices
    print("H_qp_di is positive definite.")
except np.linalg.LinAlgError:
    print("H_qp_di is NOT positive definite. Check weights (R > 0 or S > 0 if Phi is not full rank).")
    # Check eigenvalues if not PD
    eigenvalues = np.linalg.eigvalsh(H_qp_di)
    print(f"Eigenvalues of H_qp_di: {eigenvalues}")
    if np.all(eigenvalues >= 0):
        print("H_qp_di is positive semi-definite.")

Shape of H_qp_di: (10, 10)
Shape of f_qp_di: (10, 1)
H_qp_di is positive definite.


**Important Note on Control Horizon $N_c < N_p$:**

If $N_c < N_p$, the vector of decision variables $\mathbf{U}_k$ only contains the first $N_c$ moves: $\mathbf{U}_k = [u_{k|k}^T, \dots, u_{k+N_c-1|k}^T]^T$. 
The inputs beyond $N_c$ (i.e., $u_{k+N_c|k}, \dots, u_{k+N_p-1|k}$) are typically held constant, $u_{k+j|k} = u_{k+N_c-1|k}$ for $j \ge N_c$.

In this case:
1.  The $\mathbf{\Phi}$ matrix used in $\mathbf{Y}_k = \mathbf{F} x_k + \mathbf{\Phi} \mathbf{U}_k$ needs to be adjusted. It will have dimensions $N_p p \times N_c m$. Its columns correspond only to the $N_c$ free control moves, but it still predicts $N_p$ outputs. The effect of inputs held constant beyond $N_c$ would be incorporated into how this truncated $\mathbf{\Phi}$ is constructed or by adding an extra term to the free response part.
2.  The $\bar{\mathbf{R}}$ and $\bar{\mathbf{S}}$ matrices (and $\mathbf{T}_{diff}$) in the objective function would sum only up to $N_c-1$ and have dimensions corresponding to $N_c m \times N_c m$.

The `build_qp_matrices` function above assumes $N_c=N_p$ for simplicity in constructing $\bar{\mathbf{R}}$, $\bar{\mathbf{S}}$, and $\mathbf{T}_{diff}$. Adapting it for $N_c < N_p$ requires careful handling of these matrix dimensions and the structure of $\mathbf{\Phi}$.

## 4. Key Takeaways

*   The LMPC objective function quantifies the desired control performance by penalizing tracking errors and control effort/moves.
*   Weighting matrices ($\mathbf{Q}, \mathbf{R}, \mathbf{S}$) allow tuning the relative importance of these objectives.
*   The LMPC optimization problem can be transformed into a standard Quadratic Program (QP) form: $\min \frac{1}{2} \mathbf{U}^T \mathbf{H}_{QP} \mathbf{U} + \mathbf{f}_{QP}^T \mathbf{U}$.
*   The QP matrices $\mathbf{H}_{QP}$ and $\mathbf{f}_{QP}$ depend on the system model (via $\mathbf{F}, \mathbf{\Phi}$), the chosen weights, the current state $x_k$, the reference trajectory $\mathbf{R}_{traj}$, and the previously applied input $u_{k-1}$.
*   $\mathbf{H}_{QP}$ is constant (if weights are constant), while $\mathbf{f}_{QP}$ must be recomputed at each MPC iteration as $x_k$ and $\mathbf{R}_{traj}$ change.

In the next notebook (**Notebook 1.3: Solving QPs & Basic LMPC (Unconstrained)**), we will take these $\mathbf{H}_{QP}$ and $\mathbf{f}_{QP}$ matrices and use a QP solver to find the optimal control sequence $\mathbf{U}_k^*$.